In [76]:
import setup_django
setup_django.init()


In [77]:
from django.conf import settings
from ratings.models import Rating
import pandas as pd
from movies.models import Movie

LINKS_SMALL_CSV = settings.DATA_DIR / "links_small.csv"
LINKS_SMALL_CSV.exists()

True

In [78]:
qs = Rating.objects.all()
missing_movie_ids = []
for instance in qs:
    if instance.content_object is None:
        missing_movie_ids.append(instance.object_id)


_total = len(missing_movie_ids)
total_missing = list(set(missing_movie_ids))
print(len(total_missing),_total,qs.count())

3441 9462 79568


In [79]:
print(len(total_missing),_total,qs.count())

3441 9462 79568


In [80]:
links_df = pd.read_csv(LINKS_SMALL_CSV)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [81]:
ms_df = links_df.copy()[links_df.movieId.isin(total_missing)]
ms_df.head()



,movieId,imdbId,tmdbId
3,4,114885,31357.0
6,7,114319,11860.0
7,8,112302,45325.0
25,26,114057,16420.0
29,30,115012,37557.0


In [82]:
ms_df.shape[0] == len(total_missing)

True

In [83]:
def enrich_imdb_col(val):
    val = str(val)
    if len(val) == 7:
        val = f"tt{val}"
        return val
    if len(val) == 6:
        val = f"tt0{val}"
        return val
    if len(val) == 5:
        val = f"tt00{val}"
        return val
    return val
        

In [84]:
ms_df['tt'] = ms_df['imdbId'].apply(enrich_imdb_col)


In [85]:
ms_df.head()

,movieId,imdbId,tmdbId,tt
3,4,114885,31357.0,tt0114885
6,7,114319,11860.0,tt0114319
7,8,112302,45325.0,tt0112302
25,26,114057,16420.0,tt0114057
29,30,115012,37557.0,tt0115012


In [86]:
MOVIES_CSV = settings.DATA_DIR / "TMDB_movie_dataset_v11.csv"
MOVIES_CSV.exists()

True

In [87]:
df = pd.read_csv(MOVIES_CSV)

In [88]:
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"


In [89]:
movies_cols = ['title', 'overview', 'release_date','imdb_id','poster_path']

movies_df = pd.read_csv(MOVIES_CSV, usecols=movies_cols)

In [90]:
movies_df.head()

,title,release_date,imdb_id,overview,poster_path
0,Inception,2010-07-15,tt1375666,"Cobb, a skilled thief who commits corporate es...",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg
1,Interstellar,2014-11-05,tt0816692,The adventures of a group of explorers who mak...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg
2,The Dark Knight,2008-07-16,tt0468569,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg
3,Avatar,2009-12-15,tt0499549,"In the 22nd century, a paraplegic Marine is di...",/kyeqWdyUXW608qlYkRqosgbbJyK.jpg
4,The Avengers,2012-04-25,tt0848228,When an unexpected enemy emerges and threatens...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg


In [91]:
missing_movies_df = ms_df.merge(movies_df, left_on='tt', right_on = 'imdb_id')
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,title,release_date,imdb_id,overview,poster_path
0,4,114885,31357.0,tt0114885,Waiting to Exhale,1995-12-22,tt0114885,"Cheated on, mistreated and stepped on, the wom...",/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg
1,7,114319,11860.0,tt0114319,Sabrina,1995-12-15,tt0114319,An ugly duckling having undergone a remarkable...,/z1oNjotUI7D06J4LWQFQzdIuPnf.jpg
2,8,112302,45325.0,tt0112302,Tom and Huck,1995-12-22,tt0112302,"A mischievous young boy, Tom Sawyer, witnesses...",/vIG8hWOa7DyLMRiurzKwVAnIYoU.jpg
3,26,114057,16420.0,tt0114057,Othello,1995-12-15,tt0114057,The evil Iago pretends to be friend of Othello...,/nNP3FDHUIyztw5eq5AejZKzqXbz.jpg
4,30,115012,37557.0,tt0115012,Shanghai Triad,1995-09-14,tt0115012,"Shanghai, China, 1930. When young Shuisheng ar...",/tzG1UxUSfPACREjgBrsPDRUvneH.jpg


In [92]:
missing_movies_df['id'] = missing_movies_df['movieId']
missing_movies_df.dropna(subset=['tmdbId'], inplace=True)
missing_movies_df['id_alt'] = missing_movies_df['tmdbId'].apply(lambda x: str(int(x)))
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,title,release_date,imdb_id,overview,poster_path,id,id_alt
0,4,114885,31357.0,tt0114885,Waiting to Exhale,1995-12-22,tt0114885,"Cheated on, mistreated and stepped on, the wom...",/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg,4,31357
1,7,114319,11860.0,tt0114319,Sabrina,1995-12-15,tt0114319,An ugly duckling having undergone a remarkable...,/z1oNjotUI7D06J4LWQFQzdIuPnf.jpg,7,11860
2,8,112302,45325.0,tt0112302,Tom and Huck,1995-12-22,tt0112302,"A mischievous young boy, Tom Sawyer, witnesses...",/vIG8hWOa7DyLMRiurzKwVAnIYoU.jpg,8,45325
3,26,114057,16420.0,tt0114057,Othello,1995-12-15,tt0114057,The evil Iago pretends to be friend of Othello...,/nNP3FDHUIyztw5eq5AejZKzqXbz.jpg,26,16420
4,30,115012,37557.0,tt0115012,Shanghai Triad,1995-09-14,tt0115012,"Shanghai, China, 1930. When young Shuisheng ar...",/tzG1UxUSfPACREjgBrsPDRUvneH.jpg,30,37557


In [93]:
final_df = missing_movies_df.copy()[['id','id_alt','title']]
final_df['id_alt'] = final_df['id_alt'].astype(str)
final_df.head()

,id,id_alt,title
0,4,31357,Waiting to Exhale
1,7,11860,Sabrina
2,8,45325,Tom and Huck
3,26,16420,Othello
4,30,37557,Shanghai Triad


In [94]:
alt_id_list = final_df['id_alt'].to_list()

In [95]:

movies_qs = Movie.objects.filter(id__in=alt_id_list)
movies_qs.count()


160

In [96]:
from django.forms.models import model_to_dict

In [97]:
movies_qs = Movie.objects.filter(id__in=alt_id_list)
for obj in movies_qs:
    data = final_df.copy()[final_df['id_alt'] == str(obj.id)]
    if data.shape[0] == 1:
        og_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]
        if obj.title == update_data.get('title'):
            # print(og_model_data)
            og_model_data['id'] = update_data['id']
            new_model_data = {**og_model_data}
            # print(new_model_data)
            obj.delete()
            Movie.objects.create(**new_model_data)

In [98]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

Rating update took 0:00:04 (4.418568134307861s)


In [106]:
from movies.tasks import task_calculate_movie_ratings
task_calculate_movie_ratings()